In [2]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../utilities')
from basic_utilities import *
from model_utilities import *
from pandas.plotting import scatter_matrix
from ml_utilities import *
import scipy.stats as stats
import pdb
import os
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import pickle
sys.path.append('../Data_prep')
from data_config import *
sys.path.append('../../../../infrastructure/tools')
from feature_engineering import feature_engineering
from utilities import utilities
import re
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pdb

In [3]:
train_test_sample = pd.read_csv('../../../data/processed_data/train_test_sample_segment0_logit.csv')

In [4]:
train_test_sample.shape

(34338, 162)

In [5]:
get_knots = read_yaml_file('../Data_prep/1d_knots_segment0.yaml')['get_knots']

In [6]:
knots_vars = []
for var in get_knots:
    matching_strings = [s for s in train_test_sample.columns.tolist() if re.search(f"{var}_", s)]
    knots_vars += matching_strings

In [7]:
get_dummy = read_yaml_file('../Data_prep/dummy_grouping_segment0_draft.yaml')['get_dummies']

In [8]:
dummy_vars = []
for var in get_dummy:
    matching_strings = [s for s in train_test_sample.columns.tolist() if re.search(f"{var}_", s)]
    dummy_vars += matching_strings

In [9]:
binning_features = read_yaml_file('../Data_prep/binning_features_segment0.yaml')['binning_features']

In [10]:
binning_features = [f.split(',')[0] for f in binning_features]

In [11]:
bin_vars = []
for var in binning_features:
    matching_strings = [s for s in train_test_sample.columns.tolist() if re.search(f"{var}_", s)]
    bin_vars += matching_strings

In [12]:
spec1 = knots_vars + dummy_vars + bin_vars
spec1 = sorted(list(set(spec1)))

In [13]:
Xtrain = train_test_sample[spec1].astype('float')
ytrain = train_test_sample['fraud_bool']
weights = train_test_sample['sample_weight']

In [14]:
bin_vars

['bank_branch_count_8w_capped',
 'bank_branch_count_8w_0',
 'bank_branch_count_8w_1',
 'bank_branch_count_8w_2',
 'bank_branch_count_8w_bin',
 'days_since_request_capped',
 'days_since_request_0',
 'days_since_request_1',
 'days_since_request_2',
 'days_since_request_bin']

In [15]:
train_test_sample.shape

(34338, 162)

In [16]:
glm = sm.GLM(ytrain, Xtrain, family=sm.families.Binomial(), var_weights=weights)
model1 = glm.fit()
print(model1.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             fraud_bool   No. Observations:                34338
Model:                            GLM   Df Residuals:                    34281
Model Family:                Binomial   Df Model:                           56
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                    nan
Date:                Wed, 11 Jun 2025   Deviance:                   4.6936e+05
Time:                        08:25:42   Pearson chi2:                 2.30e+19
No. Iterations:                   100   Pseudo R-squ. (CS):                nan
Covariance Type:            nonrobust                                         
                                              coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [21]:
model2 = model1
while True:
    res = utilities.get_beta_n_pvalues(model2)
    spec = list(res['feature'].values)
    newspec = sorted(res.loc[(pd.notna(res['p-values'])) & (res['p-values']<0.05),'feature'])
    if len(newspec) == len(spec):
        break
    else:
        glm = sm.GLM(ytrain, Xtrain[newspec], family=sm.families.Binomial(), var_weights=weights)
        model2 = glm.fit()

In [23]:
utilities.get_beta_n_pvalues(model2)

,feature,coef,abs_coef,p-values
27,income_3,3.182236,3.182236,9.228803e-64
28,name_email_similarity_1,-2.809509,2.809509,5.643006e-113
30,name_email_similarity_4,-2.181352,2.181352,2.707614e-07
29,name_email_similarity_3,1.910844,1.910844,2.172923e-06
22,housing_status_0,-1.461645,1.461645,2.740904e-57
18,employment_status_1,1.362172,1.362172,2.641774e-02
19,employment_status_2,-1.189255,1.189255,9.097475e-09
14,device_os_0,1.188114,1.188114,2.966153e-271
25,housing_status_4,-1.046659,1.046659,3.344949e-102
16,device_os_3,0.877075,0.877075,6.513090e-42


In [25]:
print(model2.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:             fraud_bool   No. Observations:                34338
Model:                            GLM   Df Residuals:                    34296
Model Family:                Binomial   Df Model:                           41
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -20301.
Date:                Wed, 11 Jun 2025   Deviance:                       40602.
Time:                        08:28:09   Pearson chi2:                 2.86e+05
No. Iterations:                     9   Pseudo R-squ. (CS):             0.2731
Covariance Type:            nonrobust                                         
                                         coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
bank